In [1]:
just_checking_integrity=False
boundary=980000
boundary_after=-45000

In [2]:
import sys
sys.path.append("./RecSys_Course_AT_PoliMi")

In [3]:
%%capture
import numpy as np 
import pandas as pd 
import os
import scipy.sparse as sps

In [4]:
from Data_manager.split_functions.split_train_validation_random_holdout import *
from Evaluation.Evaluator import *
from Evaluation.metrics import *
from Evaluation.Evaluator import _create_empty_metrics_dict

In [5]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.NonPersonalizedRecommender import TopPop, Random, GlobalEffects
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

In [6]:
import os, multiprocessing
from functools import partial
import traceback, os
import scipy.sparse as sps

# HITRATE

In [7]:
import pandas as pd
class HITRATEEvaluator(EvaluatorHoldout):
    def __init__(self, URM_test_list, cutoff_list, min_ratings_per_user=1, exclude_seen=True,
                 diversity_object = None,
                 ignore_items = None,
                 ignore_users = None,
                 verbose=True):


        super(HITRATEEvaluator, self).__init__(URM_test_list, cutoff_list,
                                               diversity_object = diversity_object,
                                               min_ratings_per_user =min_ratings_per_user, exclude_seen=exclude_seen,
                                               ignore_items = ignore_items, ignore_users = ignore_users,
                                               verbose = verbose)
        
    def _run_evaluation_on_selected_users(self, recommender_objects, users_to_evaluate, block_size = None,items_to_compute=None,name="_"):

        if block_size is None:
            # Reduce block size if estimated memory requirement exceeds 4 GB
            block_size = min([4000, int(4*1e9*8/64/self.n_items), len(users_to_evaluate)])


        results_dict = _create_empty_metrics_dict(self.cutoff_list,
                                                  self.n_items, self.n_users,
                                                  recommender_objects[0].get_URM_train(),
                                                  self.URM_test,
                                                  self.ignore_items_ID,
                                                  self.ignore_users_ID,
                                                  self.diversity_object)


        

        # Start from -block_size to ensure it to be 0 at the first block
        user_batch_start = 0
        user_batch_end = 0

        while user_batch_start < len(users_to_evaluate):

            user_batch_end = user_batch_start + block_size
            user_batch_end = min(user_batch_end, len(users_to_evaluate))

            test_user_batch_array = np.array(users_to_evaluate[user_batch_start:user_batch_end])
            user_batch_start = user_batch_end
            recommended_items_batch_list = None
            # Compute predictions for a batch of users using vectorization, much more efficient than computing it one at a time
            for recommender_object in recommender_objects:
                recommended_items_batch_list_single, score = recommender_object.recommend(test_user_batch_array,
                                                                          remove_seen_flag=self.exclude_seen,
                                                                          cutoff = self.max_cutoff,
                                                                          items_to_compute=items_to_compute,
                                                                          remove_top_pop_flag=False,
                                                                          remove_custom_items_flag=self.ignore_items_flag,
                                                                          return_scores = True
                                                                         )
                
                if recommended_items_batch_list is None:
                    recommended_items_batch_list = recommended_items_batch_list_single
                    score_sum=score
                else:
                    recommended_items_batch_list=np.hstack((recommended_items_batch_list,recommended_items_batch_list_single))
                    score_sum+=score
                
            recommended_items_batch_list

            results_dict = self._compute_metrics_on_recommendation_list(test_user_batch_array = test_user_batch_array,
                                                         recommended_items_batch_list = recommended_items_batch_list,
                                                         score_batch=score_sum,
                                                         results_dict = results_dict,name=name)


        return results_dict
    
    def _compute_metrics_on_recommendation_list(self, test_user_batch_array, recommended_items_batch_list,score_batch,results_dict,name="_"):

        assert len(recommended_items_batch_list) == len(test_user_batch_array), "{}: recommended_items_batch_list contained recommendations for {} users, expected was {}".format(
            self.EVALUATOR_NAME, len(recommended_items_batch_list), len(test_user_batch_array))

        temp=pd.DataFrame()
        col_session=None
        col_items=None
        col_score=None
        col_max_score=None
        length=0
        # Compute recommendation quality for each user in batch
        for batch_user_index,recommended_items in enumerate(recommended_items_batch_list):
            score=score_batch[batch_user_index]
            recommended_items=np.unique(recommended_items)
            scores_top=score[recommended_items]
            
            self.len_unique+=len(recommended_items)
            test_user = test_user_batch_array[batch_user_index]
            session_id= np.repeat(test_user,len(recommended_items))
            maximum_score= np.repeat(np.max(score),len(recommended_items))
            if col_session is None:
                col_session=session_id
                col_items=recommended_items
                col_score=scores_top/np.max(score)
                col_max_score=maximum_score
            else:
                col_session=np.append(col_session,session_id)
                col_items=np.append(col_items,recommended_items)
                col_score=np.append(col_score,scores_top/np.max(score))
                col_max_score=np.append(col_max_score,maximum_score)
            #print(recommended_items[-4:])
            #print(scores_top[-4:])
            #print(session_id[-4:])
            relevant_items = self.get_user_relevant_items(test_user)

            # Being the URM CSR, the indices are the non-zero column indexes
            
            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            self._n_users_evaluated += 1

            cutoff = self.max_cutoff

            results_current_cutoff = results_dict[cutoff]
            results_current_cutoff[EvaluatorMetrics.HIT_RATE.value].add_recommendations(is_relevant)
        if time.time() - self._start_time_print > 60 or self._n_users_evaluated==len(self.users_to_evaluate):

            elapsed_time = time.time()-self._start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)

            self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Users per second: {:.0f}".format(
                          self._n_users_evaluated,
                          100.0* float(self._n_users_evaluated)/len(self.users_to_evaluate),
                          new_time_value, new_time_unit,
                          float(self._n_users_evaluated)/elapsed_time if elapsed_time>0.0 else np.nan))

            sys.stdout.flush()
            sys.stderr.flush()

            self._start_time_print = time.time()
        temp['Session_Id']=col_session
        temp['Item_ID']=col_items
        temp[f'Score_{name}']=col_score
        temp[f'Max_Score_{name}']=col_max_score
        self.dataset_for_ranker=pd.concat([self.dataset_for_ranker, temp], ignore_index=True)
        #print(temp.tail(4))
        #print(self.dataset_for_ranker.tail(4))

        return results_dict
    def evaluateRecommender(self, recommender_objects,block_size=None,items_to_compute=None,name="_"):
        """
        :param recommender_object: the trained recommender object, a BaseRecommender subclass
        :param URM_test_list: list of URMs to test the recommender against, or a single URM object
        :param cutoff_list: list of cutoffs to be use to report the scores, or a single cutoff
        :return results_df: dataframe with index the cutoff and columns the metric
        :return results_run_string: printable result string
        """
        self.len_unique=0
        if self.ignore_items_flag:
            for recommender_object in recommender_objects:
                recommender_object.set_items_to_ignore(self.ignore_items_ID)

        self._start_time = time.time()
        self._start_time_print = time.time()
        self._n_users_evaluated = 0
        self.dataset_for_ranker=pd.DataFrame(columns=['Session_Id', 'Item_ID', f'Score_{name}',f'Max_Score_{name}'])
        results_dict = self._run_evaluation_on_selected_users(recommender_objects, self.users_to_evaluate,block_size=block_size,items_to_compute=items_to_compute,name=name)
        self.dataset_for_ranker.to_csv(f"../dataset/candidates/traditional_recs/leaderboard/{name}.csv",index=False)


        if self._n_users_evaluated > 0:

            for cutoff in self.cutoff_list:
                results_current_cutoff = results_dict[cutoff]

                for key in results_current_cutoff.keys():
                    if key!="HIT_RATE":
                        continue
                    value = results_current_cutoff[key]

                    
                    results_current_cutoff[key] = value.get_metric_value()*self._n_users_evaluated
                    
                    #results_current_cutoff[key] = value/self._n_users_evaluated


        else:
            self._print("WARNING: No users had a sufficient number of relevant items")
        
        if self.ignore_items_flag:
            recommender_object.reset_items_to_ignore()

        results_df = pd.DataFrame(columns=results_dict[self.cutoff_list[0]].keys(),
                                  index=self.cutoff_list)
        results_df.index.rename("cutoff", inplace = True)

        for cutoff in results_dict.keys():
            results_df.loc[cutoff] = results_dict[cutoff]
        print("average number of different items for each session",self.len_unique/self._n_users_evaluated)
        #results_run_string = get_result_string_df(results_df)

        return results_df.loc[self.cutoff_list[0]]["HIT_RATE"]

In [8]:
from collections import defaultdict
from tqdm import tqdm
import numpy as np

from joblib import Parallel, delayed


In [9]:
import time

In [10]:
def get_ICM(files_directory="../dataset/processed_data/"):
    df_icm = pd.read_csv(filepath_or_buffer=os.path.join(files_directory, 'simplified_features_and_categories_30.csv'), sep=',', header=0)
    
    item_id_list = df_icm['item_id'].values
    feat_id_list = df_icm['feature_idx'].values
    rating_id_list = np.ones_like(feat_id_list)
    ICM_matrix = sps.csr_matrix((rating_id_list, (item_id_list, feat_id_list)))
    return ICM_matrix

In [11]:
is_leaderboard=True

In [12]:
URM_train = sps.load_npz("../dataset/processed_data/URM_WV_train_full.npz")
URM_train.data=np.ones_like(URM_train.data)
URM_valid = sps.load_npz("../dataset/processed_data/URM_WV_valid_bought.npz").tocsr()

temp= sps.load_npz("../dataset/processed_data/URM_WV_valid_seen.npz")
URM_train = temp+URM_train+URM_valid


if is_leaderboard:
    test_seen=sps.load_npz("../dataset/processed_data/URM_test_leaderboard.npz")
else:
    test_seen=sps.load_npz("../dataset/processed_data/URM_test_final.npz")
#condense matrix to just the seen users -> test = userf from 1.000.000 to 1.000.000+test_seen.shape[0]
test_seen = test_seen[test_seen.getnnz(1)>0]
if test_seen.shape[1]!=URM_train.shape[1]:
    fix=sps.csr_matrix((test_seen.shape[0],URM_train.shape[1]-test_seen.shape[1]), dtype="float32")
    test_seen=sps.hstack([test_seen,fix])
URM_after_train=sps.vstack([URM_train,test_seen]).tocsr()
M = sps.csr_matrix(URM_train.shape, dtype="float32")
URM_valid=sps.vstack([M,test_seen]).tocsr()
ICM_all = get_ICM()
    
if just_checking_integrity:
    URM_train = URM_train[boundary:]
    URM_valid = URM_valid[boundary:boundary_after]
    URM_after_train = URM_after_train[boundary:boundary_after]


#Fake URM_to recreate validation setting 





params_ICF=[
    {"topK":334,"shrink":396, "similarity":'cosine', "feature_weighting" : 'none' ,           "power":1.2561991065561426, "weight" :0.6393597471969044},
    {"topK":236,"shrink":687, "similarity":'tanimoto', "feature_weighting" : 'none' ,           "power":0.5509167891518838, "weight" :0.23362677165465845},
    {"topK":186,"shrink":699, "similarity":'dice', "feature_weighting" : 'none' ,           "power":0.5454807130915313, "weight" :0.37150060696200815},
    {"topK":236,"shrink":700, "similarity":'jaccard', "feature_weighting" : 'none' ,           "power":0.5224371929702275, "weight" :0.4532502310580744},
    {"topK":194,"shrink":45, "similarity":'adjusted', "feature_weighting" : 'none' ,           "power":1.3923780294419528, "weight" :0.747093242387402},
    {"topK":348,"shrink":661, "similarity":'asymmetric', "feature_weighting" : 'none' ,           "power":1.2462156230754484, "weight" :0.6410862143760282, "asymmetric_alpha" :0.6586444014201189},
    {"topK":587,"shrink":675, "similarity":'tversky', "feature_weighting" : 'none' ,           "power":1.894411025512657, "weight" :0.5871629850378687, "tversky_alpha" : 0.0022760544555811006,"tversky_beta" : 0.4049955813941767},
    {"topK":299,"shrink":674, "similarity":'pearson', "feature_weighting" : 'none' ,           "power":1.892818062582165, "weight" :0.5806120640896252}
]
params_graph=[
{"topK":760 , "alpha":0.35376041951192866  , "implicit":False , "power":1.6642347425902089, "weight":0.23322573709955957},
{"topK":1434 , "alpha":0.37391558455295765, "beta":0.13809420287767862  , "implicit":True , "power":1.6030310290761034, "weight":0.733513865097587}
]
params_ICBF=[
    {"topK":124,"shrink":9, "similarity":'tversky', "feature_weighting" : 'TF-IDF' ,           "power":1.861087725409631,  "tversky_alpha" : 0.7689675774356587,"tversky_beta" : 0.9052725399846582},
    {"topK":127,"shrink":79, "similarity":'asymmetric', "feature_weighting" : 'none' ,           "power":1.8699713052217346,  "asymmetric_alpha" : 0.7611612156193225},
    {"topK":218,"shrink":773, "similarity":'adjusted', "feature_weighting" : 'BM25' ,           "power":0.18601468383405495},
    {"topK":127,"shrink":19, "similarity":'cosine', "feature_weighting" : 'none' ,           "power":1.8849223054440538},
    {"topK":492,"shrink":348, "similarity":'pearson', "feature_weighting" : 'TF-IDF' ,           "power":1.467718817406988},
    {"topK":139,"shrink":3, "similarity":'dice', "feature_weighting" : 'TF-IDF' ,           "power":1.8969043493328803},
    {"topK":153,"shrink":5, "similarity":'jaccard', "feature_weighting" : 'BM25' ,           "power":1.8125421495255112},
    {"topK":152,"shrink":4, "similarity":'tanimoto', "feature_weighting" : 'BM25' ,           "power":1.8224861269207941}
]
params_UCF=[
    {"topK":1184,"shrink":386, "similarity":'cosine',           "power":0.1327615452747038,"split" :809438, "keep_dup" :False},
{"topK":1070,"shrink":15, "similarity":'tanimoto',           "power":0.845380696129735,"split" :813333, "keep_dup" :False},
{"topK":1751,"shrink":5, "similarity":'dice',           "power":0.6757609944671665,"split" :768495, "keep_dup" :False},
{"topK":1624,"shrink":16, "similarity":'jaccard',           "power":1.4426671932509196,"split" :765230, "keep_dup" :False},
{"topK":5123,"shrink":129, "similarity":'adjusted',           "power":0.12005038535002555,"split" :760859, "keep_dup" :True},
{"topK":4318,"shrink":3, "similarity":'pearson',           "power":0.16747332134568205,"split" :757664, "keep_dup" :True},
{"topK":5918,"shrink":1, "similarity":'asymmetric',           "power":1.443558847537245,"split" :776618, "keep_dup" :False,"asymmetric_alpha":0.33399818943172976},
{"topK":1031,"shrink":3, "similarity":'tversky',           "power":0.9137962256579533,"split" :775536, "keep_dup" :False,"tversky_alpha":0.5927402991066659,"tversky_beta":0.9129868678836787}
]


params=params_UCF+params_ICBF+params_ICF+params_graph

use_stacking=[False]*16+[True]*10
duplicates=[param["keep_dup"] if "keep_dup" in param else False for param in params ]
duplicates_before=[param["duplicates_train"] if "duplicates_train" in param else duplicates[i] for i,param in enumerate(params)]
duplicates_after=[param["duplicates_after_train"] if "duplicates_after_train" in param else duplicates[i] for i,param in enumerate(params)]

transpose=[True]*8+[False]*18
get_input=[lambda urm_train,urm,icm: {"URM_train":urm}]*8+[lambda urm_train,urm,icm: {"URM_train":urm,"ICM_train":icm}]*8+[lambda urm_train,urm,icm: {"URM_train":urm_train}]*10

rec_classes=([UserKNNCFRecommender]*8)+([ItemKNNCBFRecommender]*8)+([ItemKNNCFRecommender]*8)+[P3alphaRecommender,RP3betaRecommender]

recs=[]

    


In [13]:
candidates=np.array([i+1 for i in range(4990)]) #Candidates are items from 1 to 4990

In [14]:
recs_top=[]
if just_checking_integrity:
    splits=[0]
else:
    splits=[850000]
for split in splits:
    recTop=TopPop(URM_train[split:])
    recTop.fit(split)
    recTop.URM_train=URM_after_train[split:]
    recs_top.append(np.array([recTop]))
    

TopPopRecommender: URM Detected 16301 (68.8%) items with no interactions.


In [15]:
#ev = HITRATEEvaluator(URM_valid,cutoff_list=[200])
#result=ev.evaluateRecommender(recs_top[0],block_size=4000,items_to_compute=candidates,name= "tentative")

In [16]:
if not just_checking_integrity:
    URM_valid[-81618,2638]

In [17]:
for i, rec in enumerate(rec_classes):
    print(f"-------------------{i}-------------------")
    params_rec=params[i]
    params_recommender={k:v for k,v in params_rec.items()  if k not in ["weight","power","split","keep_dup","duplicates_train","duplicates_after_train"]}
    p=params_rec["power"] 
    
    split="split" in params_rec
    X=URM_train.copy()
    X_post=URM_after_train.copy()
    if not duplicates_before[i]:
        X.data=np.ones_like(X.data)
    if not duplicates_after[i]:
        X_post.data=np.ones_like(X_post.data)
    if use_stacking[i]:
        w=params_rec["weight"]
        URM_temp=sps.vstack([X*w,ICM_all.T*(1-w)]).tocsr()
        URM_temp_after=sps.vstack([X_post*w,ICM_all.T*(1-w)]).tocsr()
    else:
        URM_temp=X
        URM_temp_after=X_post
        
    if split:
        split_point=params_rec["split"]
        if just_checking_integrity:
            temp=URM_train.sum(axis=1)
        else:
            URM_temp=URM_temp[split_point:,:]
            temp=URM_train[split_point:,:].sum(axis=1)
        start_valid_user=np.sum(temp!=0)
        params_recommender["start_user"]=start_valid_user
        if not just_checking_integrity:
            URM_temp_after=URM_temp_after[split_point:,:]
    
        
    rec=rec(**get_input[i](URM_temp,URM_temp_after,ICM_all))
    rec.fit(**params_recommender)
    if split:
        if not just_checking_integrity:
            rec.offset=split_point
        else:
            rec.offset=0
    rec.W_sparse.data=np.power(rec.W_sparse.data,p)
    if transpose[i]:
        print("transposed")
        rec.W_sparse=rec.W_sparse.T
    rec.W_sparse=rec.W_sparse.astype("float32").tocsr()
    rec.URM_train=URM_temp_after.astype("float32")
    recs.append(rec)
    #rec.save_model("./",file_name =f"{rec.RECOMMENDER_NAME}-{i}.data")
recs=np.array(recs)

-------------------0-------------------
UserKNNCFRecommender: URM Detected 15123 (63.8%) items with no interactions.
Similarity column 49548 (99.1%), 449.28 column/sec. Elapsed time 1.84 min
transposed
-------------------1-------------------
UserKNNCFRecommender: URM Detected 15274 (64.5%) items with no interactions.
Similarity column 33400 (66.8%), 555.53 column/sec. Elapsed time 1.00 min
transposed
-------------------2-------------------
UserKNNCFRecommender: URM Detected 14389 (60.7%) items with no interactions.
Similarity column 27425 (54.9%), 457.08 column/sec. Elapsed time 1.00 min
transposed
-------------------3-------------------
UserKNNCFRecommender: URM Detected 14289 (60.3%) items with no interactions.
transposed
-------------------4-------------------
UserKNNCFRecommender: URM Detected 14143 (59.7%) items with no interactions.
Similarity column 23482 (47.0%), 391.31 column/sec. Elapsed time 1.00 min


/Users/nadavshaked/anaconda3/envs/RecSys_final_projrct/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in power


transposed
-------------------5-------------------
UserKNNCFRecommender: URM Detected 14027 (59.2%) items with no interactions.
Similarity column 24446 (48.9%), 404.16 column/sec. Elapsed time 1.01 min
transposed
-------------------6-------------------
UserKNNCFRecommender: URM Detected 14594 (61.6%) items with no interactions.
transposed
-------------------7-------------------
UserKNNCFRecommender: URM Detected 14576 (61.5%) items with no interactions.
Similarity column 27478 (55.0%), 457.66 column/sec. Elapsed time 1.00 min
transposed
-------------------8-------------------
ItemKNNCBFRecommender: URM Detected 9 ( 0.0%) items with no interactions.
ItemKNNCBFRecommender: ICM Detected 1 ( 0.0%) items with no features.
Similarity column 23692 (100.0%), 658.21 column/sec. Elapsed time 35.99 sec
-------------------9-------------------
ItemKNNCBFRecommender: URM Detected 9 ( 0.0%) items with no interactions.
ItemKNNCBFRecommender: ICM Detected 1 ( 0.0%) items with no features.
Similarity co

In [18]:
UCF=recs[:8]
ICBF=recs[8:16]
ICF=recs[16:24]
Graph=recs[24:]

In [19]:
if just_checking_integrity:
    rec_groups=[UCF,ICBF,ICF,Graph,*recs_top]
    names=["UCF_W","ICBF_W","ICF_W","Graph_W","TopPop15_W"]
else:
    rec_groups=[UCF,ICBF,ICF,Graph,*recs_top]
    names=["UCF_W","ICBF_W","ICF_W","Graph_W","TopPop15_W"]

In [20]:
for i,rec_group in enumerate(rec_groups):
    name=names[i]
    ev = HITRATEEvaluator(URM_valid,cutoff_list=[150])
    result=ev.evaluateRecommender(rec_group,block_size=4000,items_to_compute=candidates,name= name)

EvaluatorHoldout: Ignoring 1000000 (95.2%) Users that have less than 1 test interactions


/Users/nadavshaked/anaconda3/envs/RecSys_final_projrct/lib/python3.7/site-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in true_divide


EvaluatorHoldout: Processed 4000 ( 8.0%) in 1.20 min. Users per second: 56
EvaluatorHoldout: Processed 8000 (16.0%) in 2.26 min. Users per second: 59
EvaluatorHoldout: Processed 12000 (24.0%) in 3.32 min. Users per second: 60
EvaluatorHoldout: Processed 16000 (32.0%) in 4.33 min. Users per second: 62
EvaluatorHoldout: Processed 24000 (48.0%) in 6.31 min. Users per second: 63
EvaluatorHoldout: Processed 32000 (64.0%) in 8.34 min. Users per second: 64
EvaluatorHoldout: Processed 36000 (72.0%) in 9.40 min. Users per second: 64
EvaluatorHoldout: Processed 44000 (88.0%) in 11.44 min. Users per second: 64
EvaluatorHoldout: Processed 48000 (96.0%) in 12.46 min. Users per second: 64
EvaluatorHoldout: Processed 50000 (100.0%) in 12.83 min. Users per second: 65
average number of different items for each session 326.77116
EvaluatorHoldout: Ignoring 1000000 (95.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 4000 ( 8.0%) in 1.37 min. Users per second: 49
EvaluatorHold

OSError: [Errno 28] No space left on device